In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
import scipy
from scipy import stats
from scipy import spatial

In [2]:
path_text = "source/W2V_150.txt"
antonym_path = "source/Antonym_vietnamese.txt"
synonym_path = "source/Synonym_vietnamese.txt"

In [3]:
w2v_file = open(path_text, encoding='utf8')
antonym_file = open(antonym_path, encoding='utf8')
synonym_file = open(synonym_path, encoding='utf8')

In [4]:
words = [line for line in w2v_file.readlines()]
antonym_lines = [line for line in antonym_file.readlines()]
synonym_lines = [line for line in synonym_file.readlines()]

In [5]:
num_of_words = words[0]
vector_dimensions = words[1]
words = words[2:]

In [6]:
arr_words = []
embedding = {}

In [7]:
for idx, w in enumerate(words):
    arr_words = words[idx].split(' ')
    word = arr_words[0]
    weight = []
    for i in arr_words[2:len(arr_words)-1]:
        weight.append(float(i))
    embedding[word] = np.array(weight)

In [8]:
antonym_word1 = []
antonym_word2 = []
synonym_word1 = []
synonym_word2 = []

for idx, w in enumerate(antonym_lines):
    arr_words = antonym_lines[idx].split(' ')
    antonym_word1.append(arr_words[0])
    antonym_word2.append(arr_words[1].split('\n')[0])

for idx, w in enumerate(synonym_lines):
    arr_words = synonym_lines[idx].split(' ')
    synonym_word1.append(arr_words[0])
    synonym_word2.append(arr_words[1].split('\n')[0])

In [9]:
a = np.array([])
b = np.array([])

In [10]:
for i in range(len(antonym_word1)):
    if antonym_word1[i] in embedding and antonym_word2[i] in embedding:
        if len(a) == 0:
            a = np.append(a, embedding[antonym_word1[i]])
            b = np.append(b, embedding[antonym_word2[i]])
        else:
            a = np.vstack((a, embedding[antonym_word1[i]]))
            b = np.vstack((b, embedding[antonym_word2[i]]))

In [11]:
a1 = np.array([])
b1 = np.array([])

In [12]:
for i in range(len(synonym_word1)):
    if synonym_word1[i] in embedding and synonym_word2[i] in embedding:
        if len(a1) == 0:
            a1 = np.append(a1, embedding[synonym_word1[i]])
            b1 = np.append(b1, embedding[synonym_word2[i]])
        else:
            a1 = np.vstack((a1, embedding[synonym_word1[i]]))
            b1 = np.vstack((b1, embedding[synonym_word2[i]]))

In [13]:
y_syn = np.ones(len(a1))
y_ant = np.zeros(len(a))

In [14]:
y_train = np.concatenate((y_syn, y_ant))

In [15]:
ant_train = np.concatenate((a, b), axis=1)
syn_train = np.concatenate((a1, b1), axis=1)

In [16]:
x_train = np.concatenate((syn_train, ant_train))

clf = MLPClassifier(random_state=1, max_iter=200).fit(x_train, y_train)

In [17]:
noun_path = "source/ViData/ViCon/400_noun_pairs.txt"
adj_path = "source/ViData/ViCon/600_adj_pairs.txt"
verb_path = "source/ViData/ViCon/400_verb_pairs.txt"

In [18]:
noun_file = open(noun_path, encoding='utf8')
adj_file = open(adj_path, encoding='utf8')
verb_file = open(verb_path, encoding='utf8')

In [19]:
noun_line_test = [line for line in noun_file.readlines()]
verb_line_test = [line for line in verb_file.readlines()]
adj_line_test = [line for line in adj_file.readlines()]

In [20]:
noun_line_test = noun_line_test[1:]
verb_line_test = verb_line_test[1:]
adj_line_test = adj_line_test[1:]

In [21]:
words_test_1 = []
words_test_2 = []
label = []

for idx, w in enumerate(noun_line_test):
    words_test_1.append(noun_line_test[idx].split('\t')[0])
    words_test_2.append(noun_line_test[idx].split('\t')[1])
    if noun_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)
        
for idx, w in enumerate(verb_line_test):
    words_test_1.append(verb_line_test[idx].split('\t')[0])
    words_test_2.append(verb_line_test[idx].split('\t')[1])
    if verb_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)
        
for idx, w in enumerate(adj_line_test):
    words_test_1.append(adj_line_test[idx].split('\t')[0])
    words_test_2.append(adj_line_test[idx].split('\t')[1])
    if adj_line_test[idx].split('\t')[2].split('\n')[0] == 'ANT':
        label.append(0)
    else:
        label.append(1)

In [22]:
words_test_vt1 = np.array([])
words_test_vt2 = np.array([])
real_test_label = []

In [23]:
for i in range(len(words_test_1)):
    if words_test_1[i] in embedding and words_test_2[i] in embedding:
        if len(words_test_vt1) == 0:
            words_test_vt1 = np.append(words_test_vt1, embedding[words_test_1[i]])
            words_test_vt2 = np.append(words_test_vt2, embedding[words_test_2[i]])
            real_test_label.append(label[i])
        else:
            words_test_vt1 = np.vstack((words_test_vt1, embedding[words_test_1[i]]))
            words_test_vt2 = np.vstack((words_test_vt2, embedding[words_test_2[i]]))
            real_test_label.append(label[i])

In [24]:
test_data = np.concatenate((words_test_vt1, words_test_vt2), axis=1)

In [25]:
label_ = np.array(real_test_label)

In [26]:
LG = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(x_train, y_train)

In [27]:
lg_pred = LG.predict(test_data[:,:])

In [28]:
c = 0
for i in range(len(lg_pred)):
    if lg_pred[i] == label_[i]:
        c += 1

In [29]:
print(c/len(lg_pred))

0.6743393009377664


In [30]:
true_positive = 0
false_positive = 0
false_negative = 0
for i in range(len(lg_pred)):
    if lg_pred[i] == label_[i]:
        if lg_pred[i] == 1:
            true_positive += 1
    else:
        if label_[i] == 1:
            false_positive += 1
        else:
            false_negative += 1

In [31]:
precision = (true_positive) / (true_positive + false_positive)
recall = true_positive / (true_positive + false_negative)
F1 = 2 * ((precision * recall) / (precision + recall))

In [32]:
print(precision)
print(recall)
print(F1)

0.9194756554307116
0.591566265060241
0.7199413489736071


In [33]:
visim_path = "source/ViData/ViSim-400/Visim-400.txt"
visim_file = open(visim_path, encoding='utf8')

In [34]:
visim_line = [line for line in visim_file]

In [35]:
visim_line = visim_line[1:]

In [36]:
visim_word1 = []
visim_word2 = []

for line in visim_line:
    visim_word1.append(line.split('\t')[0])
    visim_word2.append(line.split('\t')[1])

In [37]:
def cosine_similarity(A, B):
   return np.sum(A*B) / np.sqrt(np.sum(A**2)*np.sum(B**2))

In [38]:
cosine_similarity_list = []
for i in range(len(visim_word1)):
    if visim_word1[i] in embedding and visim_word2[i] in embedding:
        cosine_similarity_list.append(cosine_similarity(embedding[visim_word1[i]], embedding[visim_word2[i]]))

In [39]:
cosine_similarity_list[1]

0.0825231832921177

In [60]:
def create_data(w1, w2):
    x = np.concatenate((embedding[w1], embedding[w2]), axis=0)
    return LG.predict(np.reshape(x, (1,300)))

In [62]:
create_data('thảnh_thơi', 'ưu_tư')

array([0.])

In [66]:
from sklearn.cluster import KMeans
X = np.array([[1, 2], [1, 4], [1, 0],
              [10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
kmeans.labels_
kmeans.predict([[0, 0], [12, 3], [0,10]])


array([1, 1, 1, 0, 0, 2])

In [ ]:
X = lis